# Poisoning Attack in Federated Learning Experiment 1.2

In [4]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime

## Google Colab

In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [3]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 4.3 MB/s 


## Additional Funtions

## MNIST(5,4)

In [ ]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [ ]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

MNIST training data loaded.
MNIST test data loaded.
Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for i in range(199):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")

old_params = copy.deepcopy(server.get_nn_parameters())
for j in range(config.CLIENTS_PER_ROUND + 1):
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

Model aggregation in round 50 was successful
Model aggregation in round 100 was successful
Train Epoch: 100 [0/300 (0%)]	Loss: 0.363989
Train Epoch: 100 [10/300 (3%)]	Loss: 0.487190
Train Epoch: 100 [20/300 (7%)]	Loss: 0.269173
Train Epoch: 100 [30/300 (10%)]	Loss: 0.369153
Train Epoch: 100 [40/300 (13%)]	Loss: 0.912088
Train Epoch: 100 [50/300 (17%)]	Loss: 0.851612
Train Epoch: 100 [60/300 (20%)]	Loss: 0.956701
Train Epoch: 100 [70/300 (23%)]	Loss: 0.298384
Train Epoch: 100 [80/300 (27%)]	Loss: 0.218178
Train Epoch: 100 [90/300 (30%)]	Loss: 0.353165
Train Epoch: 100 [100/300 (33%)]	Loss: 0.467369
Train Epoch: 100 [110/300 (37%)]	Loss: 0.394065
Train Epoch: 100 [120/300 (40%)]	Loss: 0.202010
Train Epoch: 100 [130/300 (43%)]	Loss: 0.128866
Train Epoch: 100 [140/300 (47%)]	Loss: 0.560111
Train Epoch: 100 [150/300 (50%)]	Loss: 0.068552
Train Epoch: 100 [160/300 (53%)]	Loss: 0.262426
Train Epoch: 100 [170/300 (57%)]	Loss: 0.686942
Train Epoch: 100 [180/300 (60%)]	Loss: 0.439725
Train Epoch

## FashionMNIST(5,4)


In [10]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [11]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [12]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /content/data/fmnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/data/fmnist/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /content/data/fmnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/data/fmnist/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /content/data/fmnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/fmnist/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /content/data/fmnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/fmnist/FashionMNIST/raw

FashionMnist training data loaded.
FashionMnist training data loaded.
Create 200 clients with dataset of size 300


In [13]:
import numpy as np
import copy
for i in range(199):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")

old_params = copy.deepcopy(server.get_nn_parameters())
for j in range(config.CLIENTS_PER_ROUND + 1):
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).


Model aggregation in round 50 was successful
Model aggregation in round 100 was successful
Train Epoch: 100 [0/300 (0%)]	Loss: 0.344059
Train Epoch: 100 [10/300 (3%)]	Loss: 1.366706
Train Epoch: 100 [20/300 (7%)]	Loss: 0.519858
Train Epoch: 100 [30/300 (10%)]	Loss: 1.437600
Train Epoch: 100 [40/300 (13%)]	Loss: 0.836761
Train Epoch: 100 [50/300 (17%)]	Loss: 1.016792
Train Epoch: 100 [60/300 (20%)]	Loss: 0.693168
Train Epoch: 100 [70/300 (23%)]	Loss: 0.461678
Train Epoch: 100 [80/300 (27%)]	Loss: 0.964809
Train Epoch: 100 [90/300 (30%)]	Loss: 0.943954
Train Epoch: 100 [100/300 (33%)]	Loss: 1.153473
Train Epoch: 100 [110/300 (37%)]	Loss: 1.079046
Train Epoch: 100 [120/300 (40%)]	Loss: 1.159699
Train Epoch: 100 [130/300 (43%)]	Loss: 1.218541
Train Epoch: 100 [140/300 (47%)]	Loss: 0.702101
Train Epoch: 100 [150/300 (50%)]	Loss: 0.280341
Train Epoch: 100 [160/300 (53%)]	Loss: 1.225373
Train Epoch: 100 [170/300 (57%)]	Loss: 0.487405
Train Epoch: 100 [180/300 (60%)]	Loss: 1.407409
Train Epoch